In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

In [5]:
dataset = pd.read_csv("dublinbikes_20200101_20200401.csv")
df = dataset

In [6]:
df1 = df.copy()
df1['STATUS'] = df1['STATUS'].astype('category')
df1['LAST UPDATED'] = pd.to_datetime(df1['LAST UPDATED'])
df1['TIME'] = pd.to_datetime(df1['TIME'])
df1['formatted_time'] = df1['TIME'].dt.floor('h')

df1['day_of_week'] = df1['formatted_time'].dt.strftime('%A')
df1['day_of_month'] = df1['formatted_time'].dt.strftime('%d').astype(np.int64)
df1['hour'] = df1['formatted_time'].dt.strftime('%H').astype(np.int64)
df1['month'] = df1['formatted_time'].dt.strftime('%m').astype(np.int64)
df1['week'] = df1['formatted_time'].dt.strftime('%w').astype(np.int64)
numeric_columns = df1.select_dtypes(['int64']).columns
print(df1[numeric_columns].describe().T)
print(df1.select_dtypes(['category']).describe().T)
print(df1.dtypes)

                           count       mean        std   min   25%   50%  \
STATION ID             2228278.0  60.196628  33.598271   2.0  31.0  61.0   
BIKE STANDS            2228278.0  32.184342   7.666891  16.0  29.0  30.0   
AVAILABLE BIKE STANDS  2228278.0  20.456110  11.140868   0.0  12.0  20.0   
AVAILABLE BIKES        2228278.0  11.644086   9.980968   0.0   3.0  10.0   
day_of_month           2228278.0  15.902037   9.262035   1.0   8.0  16.0   
hour                   2228278.0  11.556068   6.876381   0.0   6.0  12.0   
month                  2228278.0   2.317890   0.729879   1.0   2.0   2.0   
week                   2228278.0   2.991054   1.976522   0.0   1.0   3.0   

                        75%    max  
STATION ID             90.0  117.0  
BIKE STANDS            40.0   40.0  
AVAILABLE BIKE STANDS  29.0   41.0  
AVAILABLE BIKES        18.0   40.0  
day_of_month           24.0   31.0  
hour                   17.0   23.0  
month                   3.0    4.0  
week               

In [4]:
ts = pd.to_datetime('2020/02/01')
te = pd.to_datetime('2020/04/01')
mask = (df1['TIME'] >= ts) & (df1['TIME'] <= te)
pd.options.mode.chained_assignment = None
#suburb_point = "Merrion Square South"
#suburb_point = "Grangegorman Lower (South)"
suburb_point = "Parkgate Street"
suburb_df = df1.loc[mask]
suburb_df = suburb_df[suburb_df['ADDRESS'] == suburb_point]
suburb_dataset = suburb_df[['TIME', 'AVAILABLE BIKES']]

suburb_dataset['date'] = suburb_dataset['TIME'].dt.floor('T')
suburb_dataset = suburb_dataset.reset_index()
suburb_dataset.drop('TIME',axis=1, inplace=True)
time = suburb_dataset['date']
bikes = suburb_dataset['AVAILABLE BIKES']
#plot 2 week data for Merrion Square South region
fig = plt.figure()
ax = fig.add_subplot(111)
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.scatter(time,bikes, color="red", marker=".", label='bike count')
ax.set_xlabel("date")
ax.set_ylabel("bike available")
fig.autofmt_xdate()
ax.grid(True)
ax.set_title(f'Bike usage for {suburb_point}')
plt.show()

NameError: name 'df1' is not defined

In [193]:
# identify Ci 

import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

%matplotlib qt

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

def ridge_model(q, lag, stride):
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print(bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        XX=np.column_stack((XX,X))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)

    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    alphas = []
    coeff = []
    errors = []
    scores = []
    # Ci = 0.005
    C = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1, 10, 50, 100, 500, 1000, 2000, 5000, 10000]

    for Ci in C:
        XPoly = PolynomialFeatures(3).fit_transform(XX)
        a = 1/(2*Ci)
        print(f'Alpha & Ci  Value - {a} {Ci}')
        model = Ridge(alpha= a,max_iter=7000).fit(XPoly[train], yy[train])
        coeff.append(model.coef_)
        alphas.append(a)
        y_pred = model.predict(XPoly)
        score = model.score(XPoly[test], yy[test])
        #r2_score = r2_score(yy,y_pred)
        from sklearn.model_selection import cross_val_score
        # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
        mse = mean_squared_error(yy,y_pred)
        errors.append(np.sqrt(mse))
        print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
       .format(score,  mse,np.sqrt(mse)))
        #print(r2_score(yy,y_pred))
        r2 = r2_score(yy,y_pred)
        scores.append(r2)
        print(f'r2_score - {r2}')
    return (errors, coeff, alphas, scores)

errors, coeff, alphas, scores = ridge_model(q =2,lag = 4,stride =1)

C = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1, 10, 50, 100, 500, 1000, 2000, 5000, 10000]

plt.close("all")
fig, (ax1, ax2) = plt.subplots(2, 1)

# ax1 = plt.gca()
# ax1.ticklabel_format(useOffset=False)
alphas = [ str(i) for i in alphas ]
ax1.plot(alphas, scores)
ax1.set_xlabel("alphas")
ax1.set_ylabel("R2 Score")
ax1.set_title("alpha against R2 Score")
ax1.axis("tight")
ax1.set_xticks(alphas)

ax2 = plt.gca()
# ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax2.plot(alphas, errors)
ax2.set_xlabel("alphas")
ax2.set_ylabel("RMSE error")
ax2.set_title("Coefficient error as a function of the regularization")
ax2.axis("tight")
ax2.set_xticks(alphas)

fig.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_2808/837106973.py:11: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 4 2 7045
Alpha & Ci  Value - 5000.0 0.0001
SCORE: 0.702, MSE:12.90, RMSE:3.59
r2_score - 0.7436271760419702
Alpha & Ci  Value - 1000.0 0.0005
SCORE: 0.673, MSE:12.89, RMSE:3.59
r2_score - 0.743949013363973
Alpha & Ci  Value - 500.0 0.001
SCORE: 0.651, MSE:12.96, RMSE:3.60
r2_score - 0.7423878115887753
Alpha & Ci  Value - 100.0 0.005
SCORE: 0.615, MSE:13.16, RMSE:3.63
r2_score - 0.7385050268145834
Alpha & Ci  Value - 50.0 0.01
SCORE: 0.608, MSE:13.20, RMSE:3.63
r2_score - 0.7377076911165328
Alpha & Ci  Value - 5.0 0.1
SCORE: 0.570, MSE:13.56, RMSE:3.68
r2_score - 0.7305594203539647
Alpha & Ci  Value - 0.5 1
SCORE: 0.512, MSE:14.13, RMSE:3.76
r2_score - 0.7193080332341322
Alpha & Ci  Value - 0.05 10
SCORE: 0.500, MSE:14.25, RMSE:3.77
r2_score - 0.7168852978523772
Alpha & Ci  Value - 0.01 50
SCORE: 0.499, MSE:14.26, RMSE:3.78
r2_score - 0.7166459753736749
Alpha & Ci  Value - 0.005 100
SCORE: 0.499, MSE:14.26, RMSE:3.78
r2_score - 0.7166157549899389
Alpha & Ci  Value - 0.001 500

C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=4.3453e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.81951e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


In [200]:
# identify poly

import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

#%matplotlib inline
%matplotlib qt


t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

def ridge_model(q, lag, stride):
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print(bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        XX=np.column_stack((XX,X))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)

    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    coeff = []
    errors = []
    scores = []
    Ci = 0.001
    poly = [1,2,3,4]

    for q in poly:
        XPoly = PolynomialFeatures(q).fit_transform(XX)
        a = 1/(2*Ci)
        print(f'Alpha & Ci  Value - {a} {Ci}')
        print(f"q value - {q}")
        model = Ridge(alpha= a,max_iter=7000).fit(XPoly[train], yy[train])
        coeff.append(model.coef_)
        y_pred = model.predict(XPoly)
        score = model.score(XPoly[test], yy[test])
        #r2_score = r2_score(yy,y_pred)
        from sklearn.model_selection import cross_val_score
        # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
        mse = mean_squared_error(yy,y_pred)
        errors.append(np.sqrt(mse))
        print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
       .format(score,  mse,np.sqrt(mse)))
        #print(r2_score(yy,y_pred))
        r2 = r2_score(yy,y_pred)
        scores.append(r2)
        print(f'r2_score - {r2}')
    return (errors, coeff, alphas, scores)

errors, coeff, alphas, scores = ridge_model(q =2,lag = 4,stride =1)

poly = [1,2,3,4]

plt.close("all")
fig, (ax1, ax2) = plt.subplots(2, 1)
# ax1 = plt.gca()
# ax1.ticklabel_format(useOffset=False)
poly = [ str(i) for i in poly ]
ax1.plot(poly, scores)
ax1.set_xlabel("poly")
ax1.set_ylabel("R2 Score")
ax1.set_title("poly against R2 Score")
ax1.axis("tight")
ax1.set_xticks(poly)

ax2 = plt.gca()
# ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax2.plot(poly, errors)
ax2.set_xlabel("poly")
ax2.set_ylabel("RMSE error")
ax2.set_title("Coefficient error as a polynomial feature")
ax2.axis("tight")
ax2.set_xticks(poly)

fig.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_2808/2598805353.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 4 2 7045
Alpha & Ci  Value - 500.0 0.001
q value - 1
SCORE: 0.217, MSE:40.85, RMSE:6.39
r2_score - 0.1882356305843843
Alpha & Ci  Value - 500.0 0.001
q value - 2
SCORE: 0.506, MSE:25.61, RMSE:5.06
r2_score - 0.4910842738246871
Alpha & Ci  Value - 500.0 0.001
q value - 3
SCORE: 0.752, MSE:12.31, RMSE:3.51
r2_score - 0.7553435896004591
Alpha & Ci  Value - 500.0 0.001
q value - 4
SCORE: -2.424, MSE:38.77, RMSE:6.23
r2_score - 0.22951709147162724


In [215]:
## individual final run


import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

#%matplotlib inline
%matplotlib qt


t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

def get_feature_data(q, lag, stride, bikes, time, dt):
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print(bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        XX=np.column_stack((XX,X))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)
    return XX, yy, tt


def run_model(q, lag, stride, Ci, poly):
    plt.close("all")
    fig, ax = plt.subplots()
    XX, yy, tt = get_feature_data(q, lag, stride, bikes, time, dt)
    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    XPoly = PolynomialFeatures(poly).fit_transform(XX)
    a = 1/(2*Ci)
    print(f'Alpha & Ci  Value - {a} {Ci}')
    print(f"q value - {q}")
    model = Ridge(alpha= a,max_iter=7000).fit(XPoly[train], yy[train])
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    mse = mean_squared_error(yy,y_pred)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}" .format(score,  mse,np.sqrt(mse)))
    r2 = r2_score(yy,y_pred)
    print(f'r2_score : {r2}\n')
    ax.scatter(time, bikes, color="blue")
    ax.scatter(tt, y_pred, color="green") 
    ax.set_xlabel("time (days)")
    ax.set_ylabel("#bikes") 
    ax.set_title(f"Ridge model for q = {q}")
    ax.legend(["training data","predictions"],loc="upper right") 
    fig.show()

q = 12
lag = 4
stride = 1
Ci = 0.001
poly = 3
run_model(q, lag, stride, Ci, poly)

C:\Users\rkhna\AppData\Local\Temp/ipykernel_2808/1418064508.py:16: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 4 12 7035
Alpha & Ci  Value - 500.0 0.001
q value - 12
SCORE: 0.671, MSE:11.49, RMSE:3.39
r2_score : 0.7718248511865036



In [239]:
#final run with all identified parameters

import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

#%matplotlib inline
%matplotlib qt


t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

def get_feature_data(q, lag, stride, bikes, time, dt):
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print(bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        XX=np.column_stack((XX,X))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)
    return XX, yy, tt

qrange = [ 2, 6, 12]
lag = 4
stride = 1
plt.close("all")
fig, ax = plt.subplots(3)
ax_index = 0
Ci = 0.001
poly = 3
q_score = {}
xlim = -10
ylim = 45 # 40 for Grangegorman
for q in qrange:
    XX, yy, tt = get_feature_data(q, lag, stride, bikes, time, dt)
    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    XPoly = PolynomialFeatures(poly).fit_transform(XX)
    a = 1/(2*Ci)
    print(f'Alpha & Ci  Value - {a} {Ci}')
    print(f"q value - {q}")
    model = Ridge(alpha= a,max_iter=7000).fit(XPoly[train], yy[train])
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    mse = mean_squared_error(yy,y_pred)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}" .format(score,  mse,np.sqrt(mse)))
    r2 = r2_score(yy,y_pred)
    print(f'r2_score - {r2}\n')
    q_score[q] = r2

#     plt.close("all")
#     fig, ax = plt.subplots()

    ax[ax_index].scatter(time, bikes, color="blue")
    ax[ax_index].scatter(tt, y_pred, color="green") 
    ax[ax_index].set_xlabel("time (days)")
    ax[ax_index].set_ylabel("#bikes") 
    ax[ax_index].set_title(f"Ridge model for q = {q}")
    ax[ax_index].legend(["training data","predictions"],loc="upper right")
    ax[ax_index].set_ylim([xlim, ylim])
    ax_index += 1
print(({"-"*30}))
for q, score in q_score.items():
    print(f"q value {q} - score {score}")
fig.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_2808/1237278297.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 4 2 7045
Alpha & Ci  Value - 500.0 0.001
q value - 2
SCORE: 0.715, MSE:14.18, RMSE:3.77
r2_score - 0.7527626114904048

17127 2016 4 6 7041
Alpha & Ci  Value - 500.0 0.001
q value - 6
SCORE: 0.700, MSE:13.42, RMSE:3.66
r2_score - 0.7661062328753657

17127 2016 4 12 7035
Alpha & Ci  Value - 500.0 0.001
q value - 12
SCORE: 0.769, MSE:13.01, RMSE:3.61
r2_score - 0.7732423238240765

{'------------------------------'}
q value 2 - score 0.7527626114904048
q value 6 - score 0.7661062328753657
q value 12 - score 0.7732423238240765
